In [51]:
!pip install fasttext

In [52]:
import numpy as np

import pandas as pd

from collections import defaultdict
import os, re, csv, math, codecs
from tqdm import tqdm

import keras
import keras.backend as K
from keras.layers import Dense, GlobalAveragePooling1D, Embedding
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from sklearn import model_selection

from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import io
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer 
import os, re, csv, math, codecs
from sklearn import model_selection
from sklearn import metrics
import torch
import torch.nn as nn
import tensorflow as tf  # we use both tensorflow and pytorch (pytorch for main part) , tensorflow for tokenizer

torch.manual_seed(42);

from keras.preprocessing.text import Tokenizer
from gensim.models.fasttext import FastText
import numpy as np
import matplotlib.pyplot as plt
import nltk
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from nltk import WordPunctTokenizer

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('turkish'))

%matplotlib inline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [53]:
data=pd.read_csv("sample.csv")

In [54]:
data = data.sample(frac=1).reset_index(drop=True)


In [55]:
def sentiments(df):
  if df['yildizSayisi'] > 3.9:
    return 'positive'
  elif df['yildizSayisi'] <= 3.0:
    return 'negative'
data['sentiment'] = data.apply(sentiments, axis=1)

In [56]:
# Convert sentiment columns to numerical values
data.sentiment = data.sentiment.apply(lambda x: 1 if x=='positive' else 0)
## Cross validation 
# create new column "kfold" and assign a random value
data['kfold'] = -1
# Random the rows of data
data = data.sample(frac=1).reset_index(drop=True)
# get label
y = data.sentiment.values
# initialize kfold
kf = model_selection.StratifiedKFold(n_splits=5)
# fill the new values to kfold column
for fold, (train_, valid_) in enumerate(kf.split(X=data, y=y)):
    data.loc[valid_, 'kfold'] = fold

In [57]:
data

,yildizSayisi,yorum,yorumLike,sentiment,kfold
0,4,makinaya havlu veya fazla çamaşır attığımda ti...,0,1,0
1,3,Ürün güzel yanlız tek kötü yanı saçı çekiştirm...,0,0,0
2,5,sorunsuz çalişıyor,0,1,0
3,5,Sorunsuz hizli sekilde teslim aldim. Tasarim o...,0,1,0
4,2,Teknoloji işi şans biraz çok memnun olan arka...,1,0,0
...,...,...,...,...,...
49995,3,henüz denemedim kargo başarılıydı fakat keşke ...,0,0,4
49996,5,ürünün kendisini.ısmarladım .ama beyazı ğeldi....,0,1,4
49997,2,Fiyat gümüş bi kipe için gayet iyi ama küpe ya...,0,0,4
49998,2,Birkaç kez kullanılabiliyor hemen karardı,0,0,4


In [58]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.tr.300.vec.gz
!gunzip -k cc.tr.300.vec.gz

--2021-12-10 20:07:35--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.tr.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1261500728 (1.2G) [binary/octet-stream]
Saving to: ‘cc.tr.300.vec.gz.1’

cc.tr.300.vec.gz.1  100%[===================>]   1.17G  19.8MB/s    in 65s     

2021-12-10 20:08:40 (18.5 MB/s) - ‘cc.tr.300.vec.gz.1’ saved [1261500728/1261500728]

gzip: cc.tr.300.vec already exists; do you wish to overwrite (y or n)? y
y


In [59]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip


--2021-12-10 20:10:20--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-12-10 20:10:21--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-12-10 20:10:21--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove

In [60]:
#load fasttext embeddings
print('loading word embeddings...')
fasttext_embedding = {}
f = codecs.open('cc.tr.300.vec', encoding='utf-8')
for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    fasttext_embedding[word] = coefs
f.close()

loading word embeddings...


2000001it [04:20, 7665.32it/s]


In [61]:
fasttext_embedding['güzel'].shape

(300,)

In [62]:
glove = pd.read_csv('glove.6B.300d.txt', sep=" ", quoting=3, header=None, index_col=0)
glove_embedding = {key: val.values for key, val in glove.T.items()}

In [63]:
# Check check the dimension of this fasttext embedding version

glove_embedding['guzel'].shape

(300,)

# Building model **pipeline**

In [64]:
class hepsitrend:
    def __init__(self, reviews, targets):
        """
        Argument:
        reviews: a numpy array
        targets: a vector array
        
        Return xtrain and ylabel in torch tensor datatype, stored in dictionary format
        """
        self.reviews = reviews
        self.target = targets
    
    def __len__(self):
        # return length of dataset
        return len(self.reviews)
    
    def __getitem__(self, index):
        # given an idex (item), return review and target of that index in torch tensor
        review = torch.tensor(self.reviews[index,:], dtype = torch.long)
        target = torch.tensor(self.target[index], dtype = torch.float)
        
        return {'review': review,
                'target': target}

In [65]:
class LSTM(nn.Module):
    def __init__(self, embedding_matrix):
        """
        Given embedding_matrix: numpy array with vector for all words
        return prediction ( in torch tensor format)
        """
        super(LSTM, self).__init__()
        # Number of words = number of rows in embedding matrix
        num_words = embedding_matrix.shape[0]
        # Dimension of embedding is num of columns in the matrix
        embedding_dim = embedding_matrix.shape[1]
        # Define an input embedding layer
        self.embedding = nn.Embedding(
                                      num_embeddings=num_words,
                                      embedding_dim=embedding_dim)
        # Embedding matrix actually is collection of parameter
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype = torch.float32))
        # Because we use pretrained embedding (GLove, Fastext,etc) so we turn off requires_grad-meaning we do not train gradient on embedding weight
        self.embedding.weight.requires_grad = False
        # LSTM with hidden_size = 128
        self.lstm = nn.LSTM(
                            embedding_dim, 
                            128,
                            bidirectional=True,
                            batch_first=True,
                             )
        # Input(512) because we use bi-directional LSTM ==> hidden_size*2 + maxpooling **2  = 128*4 = 512, will be explained more on forward method
        self.out = nn.Linear(512, 1)
    def forward(self, x):
        # pass input (tokens) through embedding layer
        x = self.embedding(x)
        # fit embedding to LSTM
        hidden, _ = self.lstm(x)
        # apply mean and max pooling on lstm output
        avg_pool= torch.mean(hidden, 1)
        max_pool, index_max_pool = torch.max(hidden, 1)
        # concat avg_pool and max_pool ( so we have 256 size, also because this is bidirectional ==> 256*2 = 512)
        out = torch.cat((avg_pool, max_pool), 1)
        # fit out to self.out to conduct dimensionality reduction from 512 to 1
        out = self.out(out)
        # return output
        return out

In [66]:
def train(data_loader, model, optimizer, device):
    """
    this is model training for one epoch
    data_loader:  this is torch dataloader, just like dataset but in torch and devide into batches
    model : lstm
    optimizer : torch optimizer : adam
    device:  cuda or cpu
    """
    # set model to training mode
    model.train()
    # go through batches of data in data loader
    for data in data_loader:
        reviews = data['review']
        targets = data['target']
        # move the data to device that we want to use
        reviews = reviews.to(device, dtype = torch.long)
        targets = targets.to(device, dtype = torch.float)
        # clear the gradient
        optimizer.zero_grad()
        # make prediction from model
        predictions = model(reviews)
        # caculate the losses
        loss = nn.BCEWithLogitsLoss()(predictions, targets.view(-1,1))
        # backprob
        loss.backward()
        #single optimization step
        optimizer.step()

In [67]:
def evaluate(data_loader, model, device):
    final_predictions = []
    final_targets = []
    model.eval()
    # turn off gradient calculation
    with torch.no_grad():
        for data in data_loader:
            reviews = data['review']
            targets = data['target']
            reviews = reviews.to(device, dtype = torch.long)
            targets = targets.to(device, dtype=torch.float)
            # make prediction
            predictions = model(reviews)
            # move prediction and target to cpu
            predictions = predictions.cpu().numpy().tolist()
            targets = data['target'].cpu().numpy().tolist()
            # add predictions to final_prediction
            final_predictions.extend(predictions)
            final_targets.extend(targets)
    return final_predictions, final_targets

In [77]:
MAX_LEN = 600
TRAIN_BATCH_SIZE = 512
VALID_BATCH_SIZE = 512
EPOCHS = 10

In [78]:
def create_embedding_matrix(word_index, embedding_dict=None, d_model=100):
    """
     this function create the embedding matrix save in numpy array
    :param word_index: a dictionary with word: index_value
    :param embedding_dict: a dict with word embedding
    :d_model: the dimension of word pretrained embedding, here I just set to 100, we will define again
    :return a numpy array with embedding vectors for all known words
    """
    embedding_matrix = np.zeros((len(word_index) + 1, d_model))
    ## loop over all the words
    for word, index in word_index.items():
        if word in embedding_dict:
            embedding_matrix[index] = embedding_dict[word]
    return embedding_matrix

In [79]:
# STEP 1: Tokenization
# use tf.keras for tokenization,  
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(data.yorum.values.tolist())

In [80]:
print('Load fasttext embedding')
embedding_matrix = create_embedding_matrix(tokenizer.word_index, embedding_dict=fasttext_embedding, d_model=300)

# I just run 1 fold to reduce the time. You can try more fold to get better generalization
for fold in range(1):
    # STEP 2: cross validation
    train_df = data[data.kfold != fold].reset_index(drop=True)
    valid_df = data[data.kfold == fold].reset_index(drop=True)
    
    # STEP 3: pad sequence
    xtrain = tokenizer.texts_to_sequences(train_df.yorum.values)
    xtest = tokenizer.texts_to_sequences(valid_df.yorum.values)
    
    # zero padding
    xtrain = tf.keras.preprocessing.sequence.pad_sequences(xtrain, maxlen=MAX_LEN)
    xtest = tf.keras.preprocessing.sequence.pad_sequences(xtest, maxlen=MAX_LEN)
    
    # STEP 4: initialize dataset class for training
    train_dataset = hepsitrend(reviews=xtrain, targets=train_df.sentiment.values)
    
    # STEP 5: Load dataset to Pytorch DataLoader
    # after we have train_dataset, we create a torch dataloader to load train_dataset class based on specified batch_size
    train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size = TRAIN_BATCH_SIZE, num_workers=2)
    # initialize dataset class for validation
    valid_dataset = hepsitrend(reviews=xtest, targets=valid_df.sentiment.values)
    valid_data_loader = torch.utils.data.DataLoader(valid_dataset, batch_size = VALID_BATCH_SIZE, num_workers=1)
    
    # STEP 6: Running 
    device = torch.device('cuda')
    # feed embedding matrix to lstm
    model_fasttext = LSTM(embedding_matrix)
    # set model to cuda device
    model_fasttext.to(device)
    # initialize Adam optimizer
    optimizer = torch.optim.Adam(model_fasttext.parameters(), lr=1e-3)
    
    print('training model')
   
    for epoch in range(EPOCHS):
        #train one epoch
        train(train_data_loader, model_fasttext, optimizer,device)
        #validate
        outputs, targets = evaluate(valid_data_loader, model_fasttext,device)
        # threshold
        outputs = np.array(outputs) >= 0.5
        # calculate accuracy
        accuracy = metrics.accuracy_score(targets, outputs)
        print(f'FOLD:{fold}, epoch: {epoch}, accuracy_score: {accuracy}')

Load fasttext embedding
training model
FOLD:0, epoch: 0, accuracy_score: 0.7085
FOLD:0, epoch: 1, accuracy_score: 0.7807
FOLD:0, epoch: 2, accuracy_score: 0.8103
FOLD:0, epoch: 3, accuracy_score: 0.8123
FOLD:0, epoch: 4, accuracy_score: 0.8189
FOLD:0, epoch: 5, accuracy_score: 0.823
FOLD:0, epoch: 6, accuracy_score: 0.8347
FOLD:0, epoch: 7, accuracy_score: 0.8369
FOLD:0, epoch: 8, accuracy_score: 0.836
FOLD:0, epoch: 9, accuracy_score: 0.8394


# Glove embedding ¶

In [81]:
print('Load Glove embedding')
embedding_matrix = create_embedding_matrix(tokenizer.word_index, embedding_dict=glove_embedding, d_model=300)

for fold in range(1):
    # STEP 2: cross validation
    train_df = data[data.kfold != fold].reset_index(drop=True)
    valid_df = data[data.kfold == fold].reset_index(drop=True)
    
    # STEP 3: pad sequence
    xtrain = tokenizer.texts_to_sequences(train_df.yorum.values)
    xtest = tokenizer.texts_to_sequences(valid_df.yorum.values)
    
    # zero padding
    xtrain = tf.keras.preprocessing.sequence.pad_sequences(xtrain, maxlen=MAX_LEN)
    xtest = tf.keras.preprocessing.sequence.pad_sequences(xtest, maxlen=MAX_LEN)
    
    # STEP 4: initialize dataset class for training
    train_dataset = hepsitrend(reviews=xtrain, targets=train_df.sentiment.values)
    
    # STEP 5: Load dataset to Pytorch DataLoader
    # after we have train_dataset, we create a torch dataloader to load train_dataset class based on specified batch_size
    train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size = TRAIN_BATCH_SIZE, num_workers=2)
    # initialize dataset class for validation
    valid_dataset = hepsitrend(reviews=xtest, targets=valid_df.sentiment.values)
    valid_data_loader = torch.utils.data.DataLoader(valid_dataset, batch_size = VALID_BATCH_SIZE, num_workers=1)
    
    # STEP 6: Running 
    device = torch.device('cuda')
    # feed embedding matrix to lstm
    model_glove = LSTM(embedding_matrix)
    # set model to cuda device
    model_glove.to(device)
    # initialize Adam optimizer
    optimizer = torch.optim.Adam(model_glove.parameters(), lr=1e-3)
    
    print('training model')
   
    for epoch in range(EPOCHS):
        #train one epoch
        train(train_data_loader, model_glove, optimizer, device)
        #validate
        outputs, targets = evaluate(valid_data_loader, model_glove, device)
        # threshold
        outputs = np.array(outputs) >= 0.5
        # calculate accuracy
        accuracy = metrics.accuracy_score(targets, outputs)
        print(f'FOLD:{fold}, epoch: {epoch}, accuracy_score: {accuracy}')

Load Glove embedding
training model
FOLD:0, epoch: 0, accuracy_score: 0.6257
FOLD:0, epoch: 1, accuracy_score: 0.6449
FOLD:0, epoch: 2, accuracy_score: 0.6497
FOLD:0, epoch: 3, accuracy_score: 0.6593
FOLD:0, epoch: 4, accuracy_score: 0.6682
FOLD:0, epoch: 5, accuracy_score: 0.6678
FOLD:0, epoch: 6, accuracy_score: 0.6649
FOLD:0, epoch: 7, accuracy_score: 0.6645
FOLD:0, epoch: 8, accuracy_score: 0.6671
FOLD:0, epoch: 9, accuracy_score: 0.6706


In [83]:
def Interact_user_input(model):
    '''
    model: trained model : fasttext model or glove model
    '''
    model.eval()
    
    sentence = ''
    while True:
        try:
            sentence = input('Review: ')
            if sentence in ['q','quit']: 
                break
            sentence = np.array([sentence])
            sentence_token = tokenizer.texts_to_sequences(sentence)
            sentence_token = tf.keras.preprocessing.sequence.pad_sequences(sentence_token, maxlen = MAX_LEN)
            sentence_train = torch.tensor(sentence_token, dtype = torch.long).to(device, dtype = torch.long)
            predict = model(sentence_train)
            if predict.item() > 0.5:
                print('------> Positive')
            else:
                print('------> Negative')
        except KeyError:
            print('please enter again')

In [84]:
data

,yildizSayisi,yorum,yorumLike,sentiment,kfold
0,4,makinaya havlu veya fazla çamaşır attığımda ti...,0,1,0
1,3,Ürün güzel yanlız tek kötü yanı saçı çekiştirm...,0,0,0
2,5,sorunsuz çalişıyor,0,1,0
3,5,Sorunsuz hizli sekilde teslim aldim. Tasarim o...,0,1,0
4,2,Teknoloji işi şans biraz çok memnun olan arka...,1,0,0
...,...,...,...,...,...
49995,3,henüz denemedim kargo başarılıydı fakat keşke ...,0,0,4
49996,5,ürünün kendisini.ısmarladım .ama beyazı ğeldi....,0,1,4
49997,2,Fiyat gümüş bi kipe için gayet iyi ama küpe ya...,0,0,4
49998,2,Birkaç kez kullanılabiliyor hemen karardı,0,0,4


In [92]:
data.iloc[3]["yorum"]

'Sorunsuz hizli sekilde teslim aldim. Tasarim olarak guzel, kullanim acisindan da memnunum.'

In [93]:
Interact_user_input(model_fasttext)

Review: Sorunsuz hizli sekilde teslim aldim. Tasarim olarak guzel, kullanim acisindan da memnunum.
------> Positive
Review: q
